# Imports

## Imports and Setup

In [2]:
import pathlib
import sys
import os

# In Jupyter notebooks, __file__ is not defined
# Instead, use the current working directory to modify the path
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.insert(0, project_root)
print(f"Added {project_root} to Python path")

import torch
import numpy as np
import pyterrier as pt
from pathlib import Path

# Initialize PyTerrier
if not pt.started():
    pt.init()

from pyterrier.datasets import Dataset
from pyterrier.measures import *
from fast_forward.encoder import TASBEncoder, ContrieverEncoder
from fast_forward.index import OnDiskIndex, Mode
from fast_forward.util import Indexer
from fast_forward.util.pyterrier import FFInterpolate, FFScore
from pyterrier.terrier import Retriever

from fusions.FFTM2C2 import FFTM2C2
from fusions.experiment import fuse_convex_norm

device="cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
print(f"PyTorch version: {torch.__version__}")


Added /home/weicheng/ir-project to Python path


/tmp/ipykernel_2965/3639574956.py:18: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():
Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]
/tmp/ipykernel_2965/3639574956.py:19: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
java is now started automatically with default settings. To force initialisation early, run:
pt.java.init() # optional, forces java initialisation
  pt.init()
/home/weicheng/ir-project/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda:0
PyTorch version: 2.6.0+cu124


# Dataset Selection

In [3]:
# Dataset Selection: https://pyterrier.readthedocs.io/en/latest/datasets.html
dataset_name = "irds:beir/scifact"
dataset = pt.get_dataset(dataset_name)
testset = pt.get_dataset(dataset_name + "/test")

# Indexing
indexer = pt.IterDictIndexer(
    str(Path.cwd()),  # this will be ignored
    type=pt.index.IndexingType.MEMORY,
)
index_ref = indexer.index(dataset.get_corpus_iter(), fields=["text"])

beir/scifact documents: 100%|██████████| 5183/5183 [00:02<00:00, 2304.29it/s]


# Model Configuration

Setting up three retrieval models:
1. BM25 - Classic lexical retrieval
2. TASB - Neural retriever model
3. Contriever - Neural retriever model

In [4]:
# BM25
bm25 = pt.terrier.Retriever(index_ref, wmodel="BM25")
tasb_q_encoder =  tasb_d_encoder = TASBEncoder(device=device)
con_q_encoder = con_d_encoder = ContrieverEncoder(device=device)

In [5]:
safe_dataset_name = dataset_name.replace(":", "_").replace("/", "_")
# Define index paths for both models
tasb_index_path = Path(f"../indexes/ffindex_{safe_dataset_name}_tasb.h5")
con_index_path = Path(f"../indexes/ffindex_{safe_dataset_name}_con.h5")

def load_or_create_index(index_path: pathlib.Path, q_encoder, d_encoder):
    print(index_path.exists())
    try:
        ff_index = OnDiskIndex.load(
            index_path,
            query_encoder=q_encoder,
            mode=Mode.MAXP,
        )
    except FileNotFoundError:
        index_path.parent.mkdir(exist_ok=True, parents=True)
        ff_index = OnDiskIndex(
            index_path,
            query_encoder=q_encoder,
            mode=Mode.MAXP,
        )
        from fast_forward.util import Indexer

        def docs_iter():
            for d in dataset.get_corpus_iter():
                yield {"doc_id": d["docno"], "text": d["text"]}

        Indexer(ff_index, d_encoder, batch_size=8).from_dicts(docs_iter())

    return ff_index.to_memory()

tasb_index = load_or_create_index(tasb_index_path, tasb_q_encoder, tasb_d_encoder)
con_index = load_or_create_index(con_index_path, con_q_encoder, con_d_encoder)



True


100%|██████████| 5183/5183 [00:00<00:00, 2281120.42it/s]


True


100%|██████████| 5183/5183 [00:00<00:00, 2798542.43it/s]


## Create Retrieval Pipelines

We create three pipelines:
1. BM25 only
2. BM25 re-ranked with TASB
3. BM25 re-ranked with Contriever

In [6]:
ff_tasb = FFScore(tasb_index)
ff_con = FFScore(con_index)
RANK_CUTOFF = 50  # Number of documents to retrieve with BM25 before re-ranking

# Define retrieval pipelines
pipeline_0 = (bm25 % RANK_CUTOFF)  # BM25 only
pipeline_1 = bm25 % RANK_CUTOFF >> ff_tasb  # BM25 + TASB re-ranking
pipeline_2 = bm25 % RANK_CUTOFF >> ff_con  # BM25 + Contriever re-ranking


In [7]:

def get_pipeline_result(pipeline: Retriever, ds: Dataset):
    return pipeline.transform(ds.get_topics())

res_1 = get_pipeline_result(pipeline_0,testset)
res_2 = get_pipeline_result(pipeline_1,testset)
res_3 = get_pipeline_result(pipeline_2,testset)


13:43:36.993 [main] WARN org.terrier.querying.ApplyTermPipeline -- The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


## Model Fusion

Combine results from the three models using convex normalization,
with weights 0.2 for BM25, 0.4 for TASB, and 0.4 for Contriever.

In [8]:
fuse_min_max = fuse_convex_norm(
    df1=res_1,
    df2=res_2,
    df3=res_3,
    w1=0.2,
    w2=0.4,
    w3=0.4,
    normalization_method_1="min_max",
    normalization_method_2="min_max",
    normalization_method_3="min_max",
)

fuse_t_min_max = fuse_convex_norm(
    df1=res_1,
    df2=res_2,
    df3=res_3,
    w1=0.2,
    w2=0.4,
    w3=0.4,
    normalization_method_1="theoretical_min_max",
    normalization_method_2="theoretical_min_max",
    normalization_method_3="theoretical_min_max",
)

fuse_z_score = fuse_convex_norm(
    df1=res_1,
    df2=res_2,
    df3=res_3,
    w1=0.2,
    w2=0.4,
    w3=0.4,
    normalization_method_1="z_score",
    normalization_method_2="z_score",
    normalization_method_3="z_score",
)

fuse_min_max_lexical = fuse_convex_norm(
    df1=res_1,
    df2=res_2,
    df3=res_3,
    w1=0.2,
    w2=0.4,
    w3=0.4,
    normalization_method_1="min_max",
    normalization_method_2="unnormalized",
    normalization_method_3="unnormalized",
)

fuse_t_min_max_lexical = fuse_convex_norm(
    df1=res_1,
    df2=res_2,
    df3=res_3,
    w1=0.2,
    w2=0.4,
    w3=0.4,
    normalization_method_1="theoretical_min_max",
    normalization_method_2="unnormalized",
    normalization_method_3="unnormalized",
)

fuse_z_score_lexical = fuse_convex_norm(
    df1=res_1,
    df2=res_2,
    df3=res_3,
    w1=0.2,
    w2=0.4,
    w3=0.4,
    normalization_method_1="z_score",
    normalization_method_2="unnormalized",
    normalization_method_3="unnormalized",
)

fuse_clean = fuse_convex_norm(
    df1=res_1,
    df2=res_2,
    df3=res_3,
    w1=0.2,
    w2=0.4,
    w3=0.4,
    normalization_method_1="unnormalized",
    normalization_method_2="unnormalized",
    normalization_method_3="unnormalized",
)


/home/weicheng/ir-project/env/lib/python3.12/site-packages/pyterrier/model.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=["rank"], errors="ignore", inplace=True)
/home/weicheng/ir-project/env/lib/python3.12/site-packages/pyterrier/model.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["rank"] = df.groupby("qid", sort=False)["score"].rank(ascending=False, method="first").astype(int) -1 + FIRST_RANK
/home/weicheng/ir-project/env/lib/python3.12/site-packages/pyterrier/model.py:31: SettingWithCopyWarning: 
A value is trying to be set on 

## Evaluation Results

Compare performance of individual models vs fusion approach

In [9]:
pt.Experiment(

    [res_1,res_2,res_3,
    fuse_min_max, 
    fuse_t_min_max, 
    fuse_z_score, 
    fuse_min_max_lexical,
    fuse_t_min_max_lexical, 
    fuse_z_score_lexical,
    fuse_clean],

    testset.get_topics(),
    testset.get_qrels(),
    eval_metrics=[RR @ 10, nDCG @ 10, MAP @ 100],
    names=['bm25', 'bm25+TASB', 'bm25+Cont', 
           'min_max_fusion', 
           'theoretical_min_max_fusion', 
           'z_score_fusion', 
           'min_max_fusion_lexical_only', 
           'theoretical_min_max_fusion_lexical_only', 
           'z_score_fusion_lexical_only',
           'no_normalization_fusion'],
    baseline=0,
    correction="bonferroni",
    save_dir="./results",
)



,name,nDCG@10,AP@100,RR@10,nDCG@10 +,nDCG@10 -,nDCG@10 p-value,nDCG@10 reject,nDCG@10 p-value corrected,AP@100 +,AP@100 -,AP@100 p-value,AP@100 reject,AP@100 p-value corrected,RR@10 +,RR@10 -,RR@10 p-value,RR@10 reject,RR@10 p-value corrected
0,bm25,0.672167,0.626235,0.632427,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,False,NaN
1,bm25+TASB,0.643090,0.604246,0.611757,64.0,65.0,0.081834,False,0.736508,69.0,71.0,0.231077,False,1.000000,56.0,64.0,0.280644,False,1.000000
2,bm25+Cont,0.664668,0.609543,0.622169,67.0,66.0,0.640590,False,1.000000,70.0,67.0,0.347563,False,1.000000,62.0,65.0,0.587228,False,1.000000
3,min_max_fusion,0.690287,0.650008,0.659679,68.0,42.0,0.162625,False,1.000000,72.0,46.0,0.095701,False,0.861312,61.0,42.0,0.075014,False,0.675128
4,theoretical_min_max_fusion,0.700102,0.658921,0.666589,67.0,34.0,0.016661,False,0.149951,70.0,38.0,0.009574,False,0.086167,60.0,34.0,0.011358,False,0.102226
5,z_score_fusion,0.699981,0.659639,0.667107,67.0,33.0,0.014863,False,0.133770,71.0,36.0,0.007277,False,0.065489,60.0,33.0,0.009121,False,0.082086
6,min_max_fusion_lexical_only,0.647700,0.609164,0.616164,66.0,63.0,0.132066,False,1.000000,71.0,69.0,0.339675,False,1.000000,58.0,62.0,0.383507,False,1.000000
7,theoretical_min_max_fusion_lexical_only,0.647700,0.609164,0.616164,66.0,63.0,0.132066,False,1.000000,71.0,69.0,0.339675,False,1.000000,58.0,62.0,0.383507,False,1.000000
8,z_score_fusion_lexical_only,0.666123,0.629208,0.636914,66.0,56.0,0.686103,False,1.000000,72.0,62.0,0.857770,False,1.000000,58.0,55.0,0.795848,False,1.000000
9,no_normalization_fusion,0.689202,0.648462,0.655726,58.0,29.0,0.078741,False,0.708667,66.0,34.0,0.038208,False,0.343874,48.0,28.0,0.039319,False,0.353873


Use unnormalized fusion of 3 models as baseline here:

In [15]:
result = pt.Experiment(

    [fuse_clean,
    fuse_min_max, 
    fuse_t_min_max, 
    fuse_z_score, 
    fuse_min_max_lexical,
    fuse_t_min_max_lexical, 
    fuse_z_score_lexical,
    pipeline_0,
    pipeline_1,
    pipeline_2,
    ],

    testset.get_topics(),
    testset.get_qrels(),
    eval_metrics=[R@10, RR @ 100, nDCG @ 100, MAP @ 100],
    names=['no_normalization_fusion',
           'min_max_fusion', 
           'theoretical_min_max_fusion', 
           'z_score_fusion', 
           'min_max_fusion_lexical_only', 
           'theoretical_min_max_fusion_lexical_only', 
           'z_score_fusion_lexical_only',
           'BM25',
            'BM25+TASB',
            'BM25+Cont'
           ],
    baseline=0,
    correction="bonferroni",
    save_dir="./results",
)

result.to_csv(f'results/{safe_dataset_name}_output.csv', index=False)

result


/home/weicheng/ir-project/env/lib/python3.12/site-packages/pyterrier/pipelines.py:299: UserWarning: save_dir is set, but the file './results/BM25.res.gz' already exists. If you are aware of are happy to reuse this file to speed up evaluation, set save_mode='reuse'; if you want to overwrite it, set save_mode='overwrite'. To make this condition an error, use save_mode='error'.
  warn(("save_dir is set, but the file '%s' already exists. If you are aware of are happy to reuse this " % save_file)+
/home/weicheng/ir-project/env/lib/python3.12/site-packages/pyterrier/pipelines.py:299: UserWarning: save_dir is set, but the file './results/BM25+TASB.res.gz' already exists. If you are aware of are happy to reuse this file to speed up evaluation, set save_mode='reuse'; if you want to overwrite it, set save_mode='overwrite'. To make this condition an error, use save_mode='error'.
  warn(("save_dir is set, but the file '%s' already exists. If you are aware of are happy to reuse this " % save_file)+

,name,R@10,nDCG@100,AP@100,RR@100,R@10 +,R@10 -,R@10 p-value,R@10 reject,R@10 p-value corrected,...,AP@100 +,AP@100 -,AP@100 p-value,AP@100 reject,AP@100 p-value corrected,RR@100 +,RR@100 -,RR@100 p-value,RR@100 reject,RR@100 p-value corrected
0,no_normalization_fusion,0.813889,0.708394,0.648462,0.659882,NaN,NaN,NaN,False,NaN,...,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,False,NaN
1,min_max_fusion,0.813556,0.709600,0.650008,0.663905,7.0,7.0,0.975908,False,1.000000,...,53.0,44.0,0.856972,False,1.000000,46.0,43.0,0.668284,False,1.000000
2,theoretical_min_max_fusion,0.826778,0.716833,0.658921,0.670356,9.0,6.0,0.240236,False,1.000000,...,51.0,34.0,0.200693,False,1.000000,43.0,32.0,0.225600,False,1.000000
3,z_score_fusion,0.824111,0.717007,0.659639,0.670897,7.0,4.0,0.272013,False,1.000000,...,53.0,34.0,0.138348,False,1.000000,44.0,33.0,0.171798,False,1.000000
4,min_max_fusion_lexical_only,0.774556,0.675335,0.609164,0.621954,4.0,15.0,0.003559,True,0.032032,...,45.0,73.0,0.002872,True,0.025847,39.0,70.0,0.005117,True,0.046049
5,theoretical_min_max_fusion_lexical_only,0.774556,0.675335,0.609164,0.621954,4.0,15.0,0.003559,True,0.032032,...,45.0,73.0,0.002872,True,0.025847,39.0,70.0,0.005117,True,0.046049
6,z_score_fusion_lexical_only,0.786222,0.691125,0.629208,0.642069,3.0,11.0,0.011276,False,0.101484,...,42.0,67.0,0.088589,False,0.797297,37.0,63.0,0.128139,False,1.000000
7,BM25,0.814333,0.691317,0.626235,0.636190,9.0,13.0,0.972730,False,1.000000,...,34.0,66.0,0.038208,False,0.343874,33.0,55.0,0.032095,False,0.288853
8,BM25+TASB,0.771222,0.671378,0.604246,0.617633,4.0,16.0,0.002121,True,0.019086,...,40.0,80.0,0.001240,True,0.011159,36.0,75.0,0.002603,True,0.023427
9,BM25+Cont,0.829278,0.680282,0.609543,0.625360,15.0,12.0,0.324404,False,1.000000,...,51.0,79.0,0.010968,False,0.098712,48.0,73.0,0.028145,False,0.253308
